In [ ]:
from abc import ABC, abstractmethod
from overrides import overrides
import random
import re
random.seed(42)

In [ ]:
class UpgradeabilityPatternClassifier(BaseUPCDetector):

    @overrides
    def parse_decompiled_bytecode(self, decompiled_bytecode_lines):
        self.prepare_env()
        for curr_loc, curr_line in enumerate(decompiled_bytecode_lines):
            self.parse_contract_storage(curr_line)

    @overrides
    def is_upc(self, address, decompiled_bytecode, bytecode):
        pass
    
    def is_transparanet_upc(self, proxy_func_src):
        for line in proxy_func_src:
            if (line.find("if") >=0 and line.find("!=") < 0 and line.find("caller") >=0) or (line.find("require")>=0 and line.find("caller") >=0 and line.find("!=") >=0 ):
                return True
        return False
        
    def is_unstructured_storage(self):
        pass
    
    def detect_upc_type_under_smup(self, proxy, delegatecall):
        if not delegatecall.is_upc:
            return []
        if delegatecall.upg_ref_design != "SMUP":
            return []
        
        delegatecall_metadata = delegatecall.metadata
        upc_types = []

        impl_contract_addr_list = [addr for addr in proxy.impl_addresses if addr != "0x0000000000000000000000000000000000000000"]
        sample_impl_size = 1 #5 if len(proxy.impl_addresses) >= 5 else len(proxy.impl_addresses)
        sample_impl_addresses = random.sample(impl_contract_addr_list, sample_impl_size)
        proxy_decomp_time = decompile_contracts_in_parallel(sample_impl_addresses, bytecode_decompiler.contracts_bytecodes_hash, bytecode_decompiler.distinct_bytecodes_hash, bytecode_decompiler.decompiler_output, timeout=360)
        bytecode_decompiler.reload_decompiled_bytecodes()
        
        try:
            proxy_decompiled_bytecode_path = bytecode_decompiler.decompile_contract(proxy.addr, 1)
        except Exception as e:
            print('PROXY DECOMPILATION FAILURE')
        
        if proxy_decompiled_bytecode_path and proxy_decompiled_bytecode_path.find("Failure") < 0:
    
            with open(proxy_decompiled_bytecode_path, 'r') as file:
                # Read the contents of the file into an array
                src = file.readlines()
            if self.is_transparanet_upc(src[delegatecall_metadata[0][2]:delegatecall_metadata[0][0]]):
                upc_types.append("Transparent")        

            # if the slot is made using hashing mechanism is unstructured storage. like if we find the hashing formula within the identified implementation variable 
            elif (len(delegatecall_metadata[6])>0 and (delegatecall_metadata[6][0].find("sha") >=0 or re.search(r"([^']+\.)+", delegatecall_metadata[6][0]))):
                upc_types.append("Unstructured")
            
            # else if the slot is pre-computed 
            elif len(delegatecall_metadata[3])>0 and len(delegatecall_metadata[3][0])>30:
                if "0x360894a13ba1a3210667c828492db98dca3e2076cc3735a920a3ca505d382bbc".find(delegatecall_metadata[3][0][4:len(delegatecall_metadata[3][0])-4]) >=0:
                    upc_types.append("EIP1967")
                else:
                    upc_types.append("Unstructured")
            else:
                for impl_addr in sample_impl_addresses:
                    self.parse_decompiled_bytecode(open(bytecode_decompiler.decompile_contract(impl_addr, 1), 'r'))
                    impl_storage_layout = self.state_vars
                    proxy_storage_layout = []
                    for idx, var in enumerate(delegatecall_metadata[4]):
                        if len(self.get_variable_slot(var)) < 3 or self.get_variable_slot(var).isnumeric():
                            proxy_storage_layout.append(var)

                    if len(proxy_storage_layout) >0:
                        is_storage_layout_consistent = True
                        for proxy_var in proxy_storage_layout:
                            sw_var_persist = False
                            proxy_var_slot = self.get_variable_slot(proxy_var)
                            proxy_var_type = proxy_var[1]
                            for impl_var in impl_storage_layout:
                                if self.get_variable_slot(impl_var) == proxy_var_slot and impl_var[1] == proxy_var_type:
                                    sw_var_persist = True
                                    break
                            if not sw_var_persist:
                                is_storage_layout_consistent = False
                                break

                        if is_storage_layout_consistent:
                            upc_types.append("Inherited")
                            break
                        else:
                            if len(delegatecall_metadata[3])>0 and delegatecall_metadata[3][0].isnumeric():
                                sw_is_eternal_upc = True
                                if self.get_variable_slot(delegatecall_metadata[4][0]).isnumeric():
                                    critical_slot = int(self.get_variable_slot(delegatecall_metadata[4][0]))
                                    if critical_slot > 0:
                                        eternal_vars = []
                                        for impl_var in impl_storage_layout:
                                            impl_var_slot = self.get_variable_slot(impl_var)
                                            if impl_var_slot.isnumeric() and int(impl_var_slot) < critical_slot:
                                                eternal_vars.append(impl_var)

                                        if len(eternal_vars) > 0:
                                            sw_is_all_eternal_vars_mapping = True
                                            for eternal_var in eternal_vars:
                                                if "mapping" not in eternal_var:
                                                    sw_is_all_eternal_vars_mapping = False
                                                    break
                                            if sw_is_all_eternal_vars_mapping:
                                                upc_types.append("Eternal")
                                                break
            if len(upc_types) == 0:
                upc_types.append("SMUP Customized") # Non-pattern                         
        return upc_types
    
    def detect_upc_type_under_dup(self, proxy, delegatecall):
        if not delegatecall.is_upc:
            return []
        if delegatecall.upg_ref_design != "DUP":
            return []
        
        upc_types = []
        delegatecall_metadata = delegatecall.metadata
        if isinstance(delegatecall_metadata, str) and delegatecall_metadata.find("Diamond")>=0:
            upc_types.append("Diamond")
        else:
            sw_eip_1822 = False
            if (len(delegatecall_metadata[3])>0 and len(delegatecall_metadata[3][0])>40):
                for contract in delegatecall.contracts:
                    if len(contract.upgr_functions) > 0:
                        try:
                            with open(bytecode_decompiler.decompile_contract(contract.address, 1), 'r') as file:
                                # Read the contents of the file into an array
                                src = file.readlines()
                            for upgrade_func in contract.upgr_functions:
                                sw_proxiableUUID_call = False
                                sw_imp_slot_check = False
                                for line in src[upgrade_func[1][0]:upgrade_func[1][1]]:
                                    if line.find("call") >=0 and line.find("0x52d1902d")>=0:
                                        sw_proxiableUUID_call = True
                                    if line.find(delegatecall_metadata[3][0][4:len(delegatecall_metadata[3][0])-4])>=0:
                                        sw_imp_slot_check = True
                                if sw_proxiableUUID_call and sw_imp_slot_check:
                                    sw_eip_1822 = True
                                    break
                            if sw_eip_1822:
                                upc_types.append("EIP1822")
                                break
                        except:
                            pass
            if not sw_eip_1822:
                upc_types.append("DUP Customized") # Non-pattern
        return upc_types
    

    def detect_upc_type_under_esup(self, proxy, delegatecall):
        if not delegatecall.is_upc:
            return []
        if delegatecall.upg_ref_design != "ESUP":
            return []
        
        is_beacon = False
        is_registry = False
        upc_types = []
        for idx, ext_state_var in enumerate(delegatecall.metadata[4]):
            if self.get_variable_slot(ext_state_var) == delegatecall.metadata[3][0]:
                if "mapping" in ext_state_var or ("array" in ext_state_var and "addr" in ext_state_var):
                    is_registry = True
                    upc_types.append("Registry")
                    break
                elif "addr" in ext_state_var or ("array" in ext_state_var and "unit256" in ext_state_var):
                    is_beacon = True
                    upc_types.append("Beacon")
                    break
        if not is_beacon and not is_registry:
            upc_types.append("ESUP Customized") # Non-pattern
        return upc_types